In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("train-bank-campaign-data.csv")

In [16]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,id_var
0,56,services,married,high.school,no,no,yes,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,65790
1,45,services,married,basic.9y,unknown,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,55643
2,59,admin.,married,professional.course,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,66189
3,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,70807
4,24,technician,single,professional.course,no,yes,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,42625


In [10]:
stats = data.describe().T
stats['unique'] = [len(data[column].unique()) for column in stats.index]
stats['null'] = [data[column].isnull().sum() for column in stats.index]
stats['unknown'] = [(data[column] == "unknown").sum() for column in stats.index]
stats

,count,mean,std,min,25%,50%,75%,max,unique,null,unknown
age,37084.0,40.042714,10.432965,17.000,32.000,38.000,47.000,98.000,77,0,0
duration,37084.0,258.237946,258.730909,0.000,102.000,180.000,319.250,4918.000,1509,0,0
campaign,37084.0,2.569545,2.770611,1.000,1.000,2.000,3.000,56.000,42,0,0
pdays,37084.0,962.530849,186.773063,0.000,999.000,999.000,999.000,999.000,27,0,0
previous,37084.0,0.172986,0.495681,0.000,0.000,0.000,0.000,7.000,8,0,0
emp.var.rate,37084.0,0.082669,1.568997,-3.400,-1.800,1.100,1.400,1.400,10,0,0
cons.price.idx,37084.0,93.576076,0.578493,92.201,93.075,93.749,93.994,94.767,26,0,0
cons.conf.idx,37084.0,-40.505183,4.622045,-50.800,-42.700,-41.800,-36.400,-26.900,26,0,0
euribor3m,37084.0,3.621668,1.733972,0.634,1.344,4.857,4.961,5.045,314,0,0
nr.employed,37084.0,5167.058664,72.196605,4963.600,5099.100,5191.000,5228.100,5228.100,11,0,0


In [11]:
(data['default'] == "unknown").sum()

7700

In [12]:
cat_stats = data.describe(include=['object']).T
cat_stats['null'] = [data[column].isnull().sum() for column in cat_stats.index]
cat_stats['unknown'] = [(data[column] == "unknown").sum() for column in cat_stats.index]
cat_stats

,count,unique,top,freq,null,unknown
job,37084,12,admin.,9420,0,306
marital,37084,4,married,22479,0,72
education,37084,8,university.degree,10971,0,1549
default,37084,3,no,29382,0,7700
housing,37084,3,yes,19433,0,882
loan,37084,3,no,30561,0,882
contact,37084,2,cellular,23522,0,0
month,37084,10,may,12420,0,0
day_of_week,37084,5,thu,7778,0,0
poutcome,37084,3,nonexistent,32023,0,0


In [28]:
data['job'].value_counts()

admin.           9420
blue-collar      8314
technician       6096
services         3547
management       2637
retired          1541
entrepreneur     1309
self-employed    1276
housemaid         946
unemployed        925
student           767
unknown           306
Name: job, dtype: int64

In [27]:
data['marital'].value_counts()

married     22479
single      10407
divorced     4126
unknown        72
Name: marital, dtype: int64

In [26]:
data['education'].value_counts()

university.degree      10971
high.school             8542
basic.9y                5421
professional.course     4746
basic.4y                3765
basic.6y                2074
unknown                 1549
illiterate                16
Name: education, dtype: int64

In [29]:
data['default'].value_counts()

no         29382
unknown     7700
yes            2
Name: default, dtype: int64

What does all this information tell us?

1) There is not null values

2) We can see which columns don't contain useful information for our predictions:
* The id_var column contains a unique identifier for each row and will not be useful for prediction
* The duration column is the last contact duration. This attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

3) There are some categorical columns that contain "unknown" values. We will need to figure out how to address those. One option is to transform these features to one-hot encode (get_dummmies) and then delete the columns referring to the "unknown".

4) The column y is categorical, so we have to transform it to numerical.

4) There are some columns that are currently numeric that should be categorical (low number of unique values): previous, nr_employeed, emp_var_rate, cons.price.idx, cons.conf.idx, pdays, euriborn (numerical to categorial - ?) - Try in a second iteration.


In [31]:
data.drop(columns = 'id_var', inplace = True)

In [ ]:
target = website_dummy[['Type']]
target.head()

In [32]:
data.corr()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
age,1.000000,-0.001500,0.002777,-0.035689,0.022897,-0.000099,0.002773,0.129262,0.010845,-0.018226
duration,-0.001500,1.000000,-0.072291,-0.048254,0.022098,-0.030431,0.004068,-0.009096,-0.035182,-0.046764
campaign,0.002777,-0.072291,1.000000,0.052624,-0.078300,0.150390,0.126598,-0.012388,0.134640,0.143609
pdays,-0.035689,-0.048254,0.052624,1.000000,-0.586728,0.271053,0.078072,-0.089587,0.297038,0.372449
previous,0.022897,0.022098,-0.078300,-0.586728,1.000000,-0.418880,-0.200008,-0.052069,-0.453220,-0.500861
emp.var.rate,-0.000099,-0.030431,0.150390,0.271053,-0.418880,1.000000,0.774727,0.199778,0.972243,0.906857
cons.price.idx,0.002773,0.004068,0.126598,0.078072,-0.200008,0.774727,1.000000,0.062526,0.687433,0.521010
cons.conf.idx,0.129262,-0.009096,-0.012388,-0.089587,-0.052069,0.199778,0.062526,1.000000,0.280493,0.102931
euribor3m,0.010845,-0.035182,0.134640,0.297038,-0.453220,0.972243,0.687433,0.280493,1.000000,0.945145
nr.employed,-0.018226,-0.046764,0.143609,0.372449,-0.500861,0.906857,0.521010,0.102931,0.945145,1.000000


In [ ]:
features = ['Neighborhood', 'OverallQual', 'OverallCond', 
            'FullBath', 'HalfBath', 'BedroomAbvGr', 
            'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 
            'SalePrice', 'Total Sqft', 'Price Per Sqft']